In [1]:
import requests
import json
import time
from scrapy.http import HtmlResponse
import pandas as pd
import hashlib
from datetime import datetime,date,timedelta

In [3]:
def get_alphanumeric_list():
    alphanumeric_list = []
    for i in range(26):
        alphanumeric_list.append(chr(ord('a') + i))
    for i in range(10):
        alphanumeric_list.append(str(i))
    return alphanumeric_list
entries = get_alphanumeric_list()
print(entries)


['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [4]:
def fetch_data(searchquery,retries=2):
    list_out = []
    break_flag = False
    counter = 0
    while True:
        print(counter)
        url = "https://www.nfa.futures.org/BasicNet/basic-api/DataHandlerSearch.ashx"
        header = {
            # "Cookie":"MY_WEB_SESSION=5ccba3d86f75636729e5d5139cc28679ed08742a82752dec175fe374d00241e28fde4982; _ga=GA1.2.186092849.1687500978; _gid=GA1.2.116303700.1687500978",
        "Content-Type": "application/json; charset=UTF-8",
        # "Host":"www.nfa.futures.org",
        # "Origin":"https://www.nfa.futures.org",
        # "Referer":"https://www.nfa.futures.org/BasicNet/basic-search-results.aspx?rnd=2728.839854916077" ,
        #     "Sec-Fetch-Mode":"cors",
        #     "X-Json-Rpc": "getFirmSearchResults",
            "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.51"
        }

        pay = {
          "id": 1,
          "method": "getFirmSearchResults",
          "params": [
            searchquery,
            {
              "pageIndex": counter,
              "pageSize": 1000,
              "totalPages": 0,
              "totalCount": 0,
              "sort": [],
              "filters": {
                "memStatus": "",
                "regTypes": "",
                "regActions": ""
              },
              "filterOptions": {
                "memStatus": None,
                "regTypes": None,
                "regActions": None
              }
            }
          ]
        }
        
        for again in range(retries):
            r = requests.post(url,headers=header,json=pay)
            data = r.json()
            try:
                if len(data["result"]["result"]["result"]["rows"]):
                    counter+=1
                    
                    list_out.extend(data["result"]["result"]["result"]["rows"]) 
                else:
                    break_flag = True
                    
                break
             
            except :
                if again== retries-1:
                    print(data)
                time.sleep(60)
                continue
    
        if break_flag :
            break
    return list_out 
            
    
    

In [5]:
out_list = []
for iteration in entries:
    print(iteration)
    get_output = fetch_data(iteration)
    out_list.extend(get_output)
    

a
0
1
2
3
4
5
6
7
8
9
10
b
0
1
2
3
4
5
6
7
c
0
1
2
3
4
5
6
7
8
9
10
d
0
1
2
3
4
5
e
0
1
2
3
4
f
0
1
2
3
4
5
6
g
0
1
2
3
4
5
6
h
0
1
2
3
4
i
0
1
2
3
4
j
0
1
2
3
4
k
0
1
2
3
l
0
1
2
3
4
m
0
1
2
3
4
5
6
7
8
n
0
1
2
3
o
0
1
2
3
p
0
1
2
3
4
5
6
q
0
1
r
0
1
2
3
4
5
s
0
1
2
3
4
5
6
7
8
9
t
0
1
2
3
4
5
6
7
u
0
1
v
0
1
2
w
0
1
2
3
4
x
0
1
y
0
1
z
0
1
0
0
1
1
0
1
2
0
1
3
0
1
4
0
1
5
0
1
6
0
1
7
0
1
8
0
1
9
0
1


In [6]:
len(out_list)

108542

In [7]:
with open('Output_test.json', "w",encoding='utf-8') as outfile:
    json.dump(out_list, outfile, ensure_ascii=False, indent=4)

In [35]:
#only take value which has 'HAS_REG_ACTIONS': 'Yes'
#hash code for encrypted nfa & add same to CIN

In [18]:
file = open(r'C:\Users\abhij\Downloads\Output_test.json', 'r', encoding="utf8")
jdata = file.read()
    
obj = json.loads(jdata)

In [19]:
print(obj[1])

{'GROUP_ID': 1, 'ENTITY_ID': 'ylHhc7FF6MI=', 'FIRM_NAME': 'A', 'FIRM_NAME_DETAILS': 'NFA ID: 0547641', 'CURRENT_MEMBERSHIP_STATUS': None, 'CURRENT_REG_TYPES': ' -', 'REGULATORY_ACTIONS': None, 'HAS_REG_ACTIONS': 'No', 'PROCESSED_MEMBERSHIP_STATUS': 'Not an NFA Member', 'ENTITY_ID_DECRYPTED': None, 'ENTITY_ID_decrypted': '0547641'}


In [27]:
out_list2 = []
for i in obj:
    if i['HAS_REG_ACTIONS'] != 'No':
        name= i['FIRM_NAME']
        comments = i['FIRM_NAME_DETAILS'] + " | " + i['PROCESSED_MEMBERSHIP_STATUS']
        CIN_d = [i['ENTITY_ID_decrypted']] if i['ENTITY_ID_decrypted'] else []
        cin_e = i['ENTITY_ID_decrypted']
        
        #print(name)
#     if i['HAS_REG_ACTIONS'] not in ['Yes','No']:
#         name= i['FIRM_NAME']
#         CIN = i['ENTITY_ID_decrypted']
        #print(name )
    out_list2.append({
            "uid": get_hash(cin_e),
            "name": name,
            "alias_name":[],
            "country": [],
            "list_type": "Entity",
            "last_updated": last_updated_string,
            "entity_details": {
                "CIN":cin_e
            },
            "nns_status": False,
            "relationship_details":{} ,
            "address": [{
                "country":"",
                "complete_address": "" 
            }],
            "sanction_details": {} ,
            "documents":{
                "CIN" :CIN_d, } ,
            "comment": comments,
         
        }) 





In [28]:
with open('output_2156.json', "w",encoding='utf-8') as outfile:
    json.dump(out_list2, outfile, ensure_ascii=False, indent=4)